In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(ws)

Workspace name: udacity-project-1
Azure region: eastus
Subscription id: cab4bd7f-364d-4fe9-a4fe-ef2652a1633e
Resource group: ownresources
Workspace.create(name='udacity-project-1', subscription_id='cab4bd7f-364d-4fe9-a4fe-ef2652a1633e', resource_group='ownresources')


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

    # Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())    

Creating a new compute cluster...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-13T21:50:44.666000+00:00', 'errors': None, 'creationTime': '2021-03-13T21:50:39.828459+00:00', 'modifiedTime': '2021-03-13T21:50:56.619009+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
# from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e?wsid=/subscriptions/cab4bd7f-364d-4fe9-a4fe-ef2652a1633e/resourcegroups/ownresources/workspaces/udacity-project-1

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-13T21:51:01.626665][API][INFO]Experiment created<END>\n""<START>[2021-03-13T21:51:02.407611][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-13T21:51:02.874224][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-13T21:51:03.3601991Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e?wsid=/subscriptions/cab4bd7f-364d-4fe9-a

In [5]:
import joblib
# Get your best run and save the model from that run.

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# Get your best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
# get_file_names()
# Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())
# log_accuracy_table()
# Logs an accuracy table to the artifact store. Maybe use it for testing.
# log_accuracy_table(name, value, description='')

[{'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_0', 'hyperparameters': None, 'best_primary_metric': 0.9176024279210926, 'status': 'Completed'}, {'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_7', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_2', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_10', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_8', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_6', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_127140f8-6f8f-4917-8cd5-ebdd3e3c465e_5', 'hyperparameters': None, 'best_primary_metric': 0.9163

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= ds,
    label_column_name= 'y',
    n_cross_validations=5)

In [9]:
# Submit your automl run

automlrun = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_76196bf5-5796-412f-b294-f0e4275a60ff

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [12]:
# Retrieve and save your best automl model.

atml_best_run = automlrun.get_best_child()

atml_best_run.get_details

<bound method Run.get_details of Run(Experiment: udacity-project,
Id: AutoML_76196bf5-5796-412f-b294-f0e4275a60ff_44,
Type: None,
Status: Completed)>

In [11]:
# Cluster clean up

compute_target.delete()